<a href="https://colab.research.google.com/github/hyo250/gittest/blob/master/%EC%84%9C%EC%9A%B8%EC%8B%9C_%EB%AA%A8%EA%B8%B0%EB%B0%9C%EC%83%9D%EC%83%81%ED%99%A9_%EC%A7%80%ED%91%9C_%EC%98%88%EC%B8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
train_x =pd.read_csv('https://raw.githubusercontent.com/Datamanim/mosquito/main/train_x.csv',encoding='euc-kr')
train_y =pd.read_csv('https://raw.githubusercontent.com/Datamanim/mosquito/main/train_y.csv',encoding='euc-kr')
test_x =pd.read_csv('https://raw.githubusercontent.com/Datamanim/mosquito/main/test_x.csv',encoding='euc-kr')
sub    =pd.read_csv('https://raw.githubusercontent.com/Datamanim/mosquito/main/sub.csv')

In [4]:
#@title  2016년~ 2019년까지의 일별 모기지수 데이터를 온도,강수량 데이터를 통해 예측해본다. 평가지표는 r2 score


In [5]:
train_x.head()

,date,강수량(mm),평균기온(℃),최저기온(℃),최고기온(℃)
0,2019-12-31,0.0,-7.9,-10.9,-4.5
1,2019-12-30,0.4,2.7,-5.7,6.8
2,2019-12-29,1.4,3.8,1.1,6.2
3,2019-12-27,0.0,-1.7,-4.6,2.6
4,2019-12-25,0.0,2.0,-2.7,6.6


In [6]:
train_y.head()

,date,mosquito_ratio
0,2019-12-31,5.5
1,2019-12-30,5.5
2,2019-12-29,5.5
3,2019-12-27,5.5
4,2019-12-25,5.5


In [8]:
test_x.head()

,date,강수량(mm),평균기온(℃),최저기온(℃),최고기온(℃)
0,2016-05-01,0.0,18.8,12.2,26.0
1,2016-05-25,0.0,19.7,15.3,24.4
2,2016-06-06,0.0,23.3,20.8,28.4
3,2016-06-09,0.0,24.1,17.8,31.3
4,2016-06-10,0.0,24.6,20.2,31.3


In [16]:
pd.options.display.max_rows

60

In [18]:
sub #값이 비어있음.

,date,mosquito_ratio
0,2016-05-01,0
1,2016-05-25,0
2,2016-06-06,0
3,2016-06-09,0
4,2016-06-10,0
...,...,...
295,2019-12-14,0
296,2019-12-19,0
297,2019-12-23,0
298,2019-12-26,0


In [11]:
train=pd.merge(train_x, train_y)
train.head()

,date,강수량(mm),평균기온(℃),최저기온(℃),최고기온(℃),mosquito_ratio
0,2019-12-31,0.0,-7.9,-10.9,-4.5,5.5
1,2019-12-30,0.4,2.7,-5.7,6.8,5.5
2,2019-12-29,1.4,3.8,1.1,6.2,5.5
3,2019-12-27,0.0,-1.7,-4.6,2.6,5.5
4,2019-12-25,0.0,2.0,-2.7,6.6,5.5


In [ ]:
#정답 원본
# from sklearn.ensemble import RandomForestRegressor
# from xgboost import XGBRFRegressor
# from sklearn.metrics import r2_score
# from sklearn.model_selection import train_test_split

# def preprocessing(df):
#     df['date'] = pd.to_datetime(df['date'])
#     df['year']= df['date'].dt.year
#     df['month']= df['date'].dt.month
#     df=df.drop(['date'],axis=1)
#     return df

# x = preprocessing(train_x)
# y = train_y.drop(['date'],axis=1)

# rf = RandomForestRegressor(random_state =12)
# xg =XGBRFRegressor(random_state =12)
# xtr ,xt, ytr,yt = train_test_split(x,y,test_size=0.3,random_state=24)

# rf.fit(xtr,ytr.values.ravel())
# xg.fit(xtr,ytr)
# pred= rf.predict(xt)
# predxg= xg.predict(xt)

# Ans = 'randomforest r2 : '+str(r2_score(yt,pred))+' \nxgboost r2 : '+str(r2_score(yt,predxg))


# subDF = preprocessing(test_x)

# pred = (rf.predict(subDF) + xg.predict(subDF))/2
# sub['mosquito_ratio'] = pred
# sub.to_csv('submission.csv',index=False)

In [14]:
#정답코드
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRFRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

def preprocessing(df): #데이터프레임을 인수로 받는 함수
    df['date'] = pd.to_datetime(df['date']) #date컬럼을 날짜형 데이터화
    df['year']= df['date'].dt.year #'year' 컬럼 생성
    df['month']= df['date'].dt.month #'month' 컬럼 생성
    df=df.drop(['date'],axis=1) #원래 데이터에서 date는 빼버림
    return df #위의 데이터를 반환.

In [15]:
x = preprocessing(train_x) #함수에 df로  train_x를 넣음. 이걸 x로 선언.
y = train_y.drop(['date'],axis=1) #y데이터에는 'date'컬럼을 뺀 것을 넣음(x데이터에 연도, 월별 컬럼이 있으니까.)

rf = RandomForestRegressor(random_state =100) #랜덤포레스트회귀모델(모두 연속형 변수). 씨드를 설정.
xg =XGBRFRegressor(random_state =100) #XGBRFR 모델 설정.
X_train ,X_test, y_train,y_test = train_test_split(x, y, test_size=0.3,random_state=100) #7대 3으로 학습용 테스트용 설정. 

rf.fit(X_train, y_train.values.ravel())
xg.fit(X_train,y_train)
pred= rf.predict(X_test) 
predxg= xg.predict(X_test)
Ans = 'randomforest r2 : '+str(r2_score(y_test, pred))+' \nxgboost r2 : '+str(r2_score(y_test,predxg))
print(Ans)

[11:46:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
randomforest r2 : 0.8494169141716839 
xgboost r2 : 0.7917612642230105


In [19]:
subDF = preprocessing(test_x) #맨위에 따로 떼어놓은 test_x 데이터 위의 X_test와 헷갈리지 말것.

pred = (rf.predict(subDF) + xg.predict(subDF))/2 ##이건 왜?
sub['mosquito_ratio'] = pred #sub에 mosquito_ration가 있었지만 0ㅇ로 채워진 컬럼이었음. 위의 test_x를 넣고 예측한 값이 여기에 해당하겠지. 이걸로 모델의 정확성을 측정
sub.to_csv('submission.csv',index=False)

In [20]:
#제출 코드 확인
def FinalMseScore():
    import pandas as pd
    y_true = pd.read_csv("https://raw.githubusercontent.com/Datamanim/mosquito/main/result.csv") #정답 데이터.
    sub = pd.read_csv('./submission.csv')
    pred = sub.iloc[:,-1].values #sub의 mosquitio_ratio만 가져옴.
    
    from sklearn.metrics import r2_score
    mse = r2_score(y_true['mosquito_ratio'],pred)    
    print('submission mse score : ',mse)
    return mse

In [21]:
final_mse = FinalMseScore()

submission mse score :  0.8395864745195029


In [22]:
# #https://colab.research.google.com/drive/1z2ri4VWYuM16ZZxUYK3rwHOgcs3t-34X?usp=sharing#scrollTo=Q4vQbv-ds32s
# 이거 보고 내가 해보기

In [26]:
from xgboost import XGBRFRegressor
from sklearn.metrics import r2_score, accuracy_score
import sklearn.metrics
dir(sklearn.metrics)

['ConfusionMatrixDisplay',
 'PrecisionRecallDisplay',
 'RocCurveDisplay',
 'SCORERS',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_base',
 '_classification',
 '_pairwise_fast',
 '_plot',
 '_ranking',
 '_regression',
 '_scorer',
 'accuracy_score',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'auc',
 'average_precision_score',
 'balanced_accuracy_score',
 'brier_score_loss',
 'calinski_harabasz_score',
 'calinski_harabaz_score',
 'check_scoring',
 'classification_report',
 'cluster',
 'cohen_kappa_score',
 'completeness_score',
 'confusion_matrix',
 'consensus_score',
 'coverage_error',
 'davies_bouldin_score',
 'dcg_score',
 'euclidean_distances',
 'explained_variance_score',
 'f1_score',
 'fbeta_score',
 'fowlkes_mallows_score',
 'get_scorer',
 'hamming_loss',
 'hinge_loss',
 'homogeneity_completeness_v_measure',
 'homogeneity_score',
 'jaccard_score',
 'jaccard_similarity_score'